In [1]:
import numpy as np
import torch

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_dataset

In [2]:
# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Load dataset
dataset = load_dataset("Hate-speech-CNERG/hatexplain", trust_remote_code=True)

In [4]:
# Process the dataset
def process_example(example):
    # Use majority voting for the label
    label_counts = {}
    for label in example['annotators']['label']:
        label_counts[label] = label_counts.get(label, 0) + 1
    majority_label = max(label_counts, key=label_counts.get)
    
    # Combine tokens into a single string
    text = " ".join(example['post_tokens'])
    return {"text": text, "label": majority_label}

In [5]:
# Apply processing to the dataset
processed_dataset = dataset.map(process_example, remove_columns=dataset['train'].column_names)

In [6]:
# Inspect the processed dataset
print(processed_dataset['train'][0])

{'text': 'u really think i would not have been raped by feral hindu or muslim back in india or bangladesh and a neo nazi would rape me as well just to see me cry', 'label': 2}


In [7]:
# Tokenizer setup
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

# Tokenize the dataset
tokenized_datasets = processed_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.with_format("torch")

In [8]:
# Split datasets
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]
label_list = sorted(set(processed_dataset["train"]["label"]))

In [9]:
# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_list))
model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Define evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # If logits are NumPy arrays:
    # predictions = np.argmax(logits, axis=-1)

    # If logits might be PyTorch tensors, convert them to NumPy:
    # (But usually, Hugging Face Trainer sends them as NumPy already)
    if isinstance(logits, torch.Tensor):
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()
    else:
        predictions = np.argmax(logits, axis=-1)

    # labels is already a NumPy array, so we can directly use it
    # If needed, ensure it's NumPy:
    # if isinstance(labels, torch.Tensor):
    #     labels = labels.cpu().numpy()

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

In [11]:
# Set up TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated from evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,  # Optional for best model selection
    fp16=torch.cuda.is_available(),  # Enable mixed precision if GPU is available
)

In [12]:
# Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_89508/1331462861.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [13]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.710700,0.728986,0.687110,0.668233,0.683775,0.687110
2,0.638300,0.709807,0.697505,0.689981,0.687982,0.697505
3,0.583100,0.773079,0.690748,0.685066,0.683801,0.690748


TrainOutput(global_step=2886, training_loss=0.6437207944510467, metrics={'train_runtime': 456.9723, 'train_samples_per_second': 100.989, 'train_steps_per_second': 6.315, 'total_flos': 1.2142421114729472e+16, 'train_loss': 0.6437207944510467, 'epoch': 3.0})

In [14]:
# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.709806501865387, 'eval_accuracy': 0.6975051975051975, 'eval_f1': 0.6899809740684529, 'eval_precision': 0.6879815951164376, 'eval_recall': 0.6975051975051975, 'eval_runtime': 5.0373, 'eval_samples_per_second': 381.953, 'eval_steps_per_second': 24.021, 'epoch': 3.0}


In [15]:
# Save the model
model.save_pretrained("./hate-speech-bert")
tokenizer.save_pretrained("./hate-speech-bert")

('./hate-speech-bert/tokenizer_config.json',
 './hate-speech-bert/special_tokens_map.json',
 './hate-speech-bert/vocab.txt',
 './hate-speech-bert/added_tokens.json')